In [1]:
import mysql.connector
import numpy as np
import pandas as pd

In [2]:
!pip install sqlalchemy pymysql

In [3]:
from sqlalchemy import create_engine

# Replace the values with your database connection details
username = 'root'
password = 'Plokij2511!'
host = 'localhost'
port = '3306'  # Default MySQL port
database = 'campusx'

# Create an engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

#### 1. Your Team have to promote shark Tank India season 4, The senior come up with the idea to show highest funding domain wise and you were assigned the task to show the same.

In [4]:
pd.read_sql_query('select industry, max(Total_Deal_Amount_in_lakhs) as Total_deal_in_lakhs from sharktank group by industry order by Total_deal_in_lakhs desc',engine)

,industry,Total_deal_in_lakhs
0,Beauty/Fashion,300.0
1,Medical/Health,250.0
2,Food,200.0
3,Manufacturing,200.0
4,Technology/Software,200.0
5,Electronics,200.0
6,Liquor/Beverages,200.0
7,Education,150.0
8,Services,150.0
9,Entertainment,150.0


#### 2. You have been assigned the role of finding the domain where female as pitchers have female to male pitcher ratio >70%

In [5]:
pd.read_sql_query('select *, round(female/male*100,2) as percentage from (select industry, sum(Female_Presenters) as female, sum(Male_Presenters) as male from sharktank group by industry having female>0 and male>0) a having percentage>70',engine)

,industry,female,male,percentage
0,Beauty/Fashion,77.0,90.0,85.56
1,Education,11.0,15.0,73.33
2,Animal/Pets,5.0,7.0,71.43


#### 3. You are working at marketing firm of Shark Tank India, you have got the task to determine volume of per year sale pitch made, pitches who received offer and pitches that were converted. Also show the percentage of pitches converted and percentage of pitches received.

In [6]:
pd.read_sql_query('select a.Season_Number, Total, Received, round((Received/Total)*100,2) as Perc_received, Accepted, round((Accepted/Total)*100,2) as Perc_Accepted from (\
select Season_Number, count(Startup_Name) as Total from sharktank group by Season_Number) a \
inner join ( \
select Season_Number, count(Startup_Name) as Received from sharktank where Received_Offer="Yes" group by Season_Number) b \
on a.Season_Number=b.Season_Number inner join ( \
select Season_Number, count(Startup_Name) as Accepted from sharktank where Accepted_Offer="Yes" group by Season_Number) c \
on b.Season_Number=c.Season_Number',engine)

,Season_Number,Total,Received,Perc_received,Accepted,Perc_Accepted
0,1,152,96,63.16,70,46.05
1,2,169,121,71.60,106,62.72
2,3,157,104,66.24,92,58.60


#### 4. As a venture capital firm specializing in investing in startups featured on a renowned entrepreneurship TV show, how would you determine the season with the startups with highest average monthly sales and identify the top 5 industries with the highest average monthly sales during that season to optimize investment decisions?

In [7]:
pd.read_sql_query('select Industry, round(avg(Monthly_Sales_in_lakhs),2) avrg from sharktank where Season_Number=(select Season_Number from (\
select Season_Number, round(avg(Monthly_Sales_in_lakhs),2) avrg from sharktank group by Season_Number order by avrg desc limit 1) a) \
group by Industry order by avrg desc limit 5',engine)

,Industry,avrg
0,Electronics,3500.00
1,Furnishing/Household,96.27
2,Beauty/Fashion,61.65
3,Food,41.28
4,Services,29.00


#### 5. As a data scientist at our firm, your role involves solving real-world challenges like identifying industries with consistent increases in funds raised over multiple seasons. This requires focusing on industries where data is available across all three years.  Once these industries are pinpointed, your task is to delve into the specifics, analyzing the number of pitches made, offers received, and offers converted per season within each industry.

In [8]:
pd.read_sql_query('with cte as (select Industry,\
max(case when Season_Number=1 then Total_Deal_Amount_in_lakhs end) as S1,\
max(case when Season_Number=2 then Total_Deal_Amount_in_lakhs end) as S2,\
max(case when Season_Number=3 then Total_Deal_Amount_in_lakhs end) as S3 from sharktank group by Industry having S2>S1 and S3>S2 and S1 != 0) \
select s.Industry,s.Season_Number, count(s.Startup_Name) as total, \
count(case when s.Received_Offer="Yes" then s.Startup_Name end) as Received, \
count(case when s.Accepted_Offer="Yes" then s.Startup_Name end) as Accepted \
from cte c inner join sharktank s on c.Industry=s.Industry \
group by s.Industry, s.Season_Number \
order by s.Industry',engine)

,Industry,Season_Number,total,Received,Accepted
0,Agriculture,1,2,1,1
1,Agriculture,2,1,1,1
2,Agriculture,3,1,1,1
3,Beauty/Fashion,1,26,17,14
4,Beauty/Fashion,2,31,24,20
5,Beauty/Fashion,3,38,25,20
6,Medical/Health,1,9,7,5
7,Medical/Health,2,15,14,13
8,Medical/Health,3,13,12,12


#### 6. Every shark want to  know in how much year their investment will be returned, so you have to create a system for them , where shark will enter the name of the startup's  and the based on the total deal and quity given in how many years their principal amount will be returned. 

In [9]:
#Delimiter \\
#create procedure TOT(in startup varchar(255))
#begin
#	case when (select Accepted_Offer="No" from sharktank where Startup_Name=startup)
#		then select "TOT cannot be calculated as offer wasn't accepted";
#		when (select Accepted_Offer="Yes" and Yearly_Revenue_in_lakhs="Not Mentioned" from sharktank where Startup_Name=startup)
#		then select "TOT cannot be calculated due to insufficient data";
#	else
#		select `Startup_Name`,`Yearly_Revenue_in_lakhs`, `Total_Deal_Amount_in_lakhs`, `Total_Deal_Equity_%`,
#        `Total_Deal_Amount_in_lakhs`/(`Total_Deal_Equity_%`*100)*`Yearly_Revenue_in_lakhs` as Years_to_return from sharktank
#        where Startup_Name=startup;
#	end case;
#end \\
#Delimiter ;


In [10]:
pd.read_sql_query('call TOT("BluePineFoods")',engine)

,Startup_Name,Yearly_Revenue_in_lakhs,Total_Deal_Amount_in_lakhs,Total_Deal_Equity_%,Years_to_return
0,BluePineFoods,95.0,75.0,16.0,4.453125


#### 7. In the world of startup investing, we're curious to know which big-name investor, often referred to as "sharks," tends to put the most money into each deal on average. This comparison helps us see who's the most generous with their investments and how they measure up against their fellow investors.


In [11]:
pd.read_sql_query('select Shark, round(avg(investment),2) as Avrg_in_lakhs from ( \
select `Namita_Investment_Amount_in_lakhs` as investment, "Namita" as Shark from sharktank where Namita_Investment_Amount_in_lakhs>0 \
union all \
select `Vineeta_Investment_Amount_in_lakhs` as investment, "Vineeta" as Shark from sharktank where Vineeta_Investment_Amount_in_lakhs>0 \
union all \
select `Anupam_Investment_Amount_in_lakhs` as investment, "Anupam" as Shark from sharktank where Anupam_Investment_Amount_in_lakhs>0 \
union all \
select `Aman_Investment_Amount_in_lakhs` as investment, "Aman" as Shark from sharktank where Aman_Investment_Amount_in_lakhs>0 \
union all \
select `Peyush_Investment_Amount__in_lakhs` as investment, "Peyush" as Shark from sharktank where Peyush_Investment_Amount__in_lakhs>0 \
union all \
select `Amit_Investment_Amount_in_lakhs` as investment, "Amit" as Shark from sharktank where Amit_Investment_Amount_in_lakhs>0 \
union all \
select `Ashneer_Investment_Amount` as investment, "Ashneer" as Shark from sharktank where Ashneer_Investment_Amount>0) k \
group by Shark',engine)

,Shark,Avrg_in_lakhs
0,Namita,32.94
1,Vineeta,31.25
2,Anupam,29.99
3,Aman,34.18
4,Peyush,35.06
5,Amit,35.27
6,Ashneer,25.68


#### 8. Develop a system that accepts inputs for the season number and the name of a shark. The procedure will then provide detailed insights into the total investment made by that specific shark across different industries during the specified season. Additionally, it will calculate the percentage of their investment in each sector relative to the total investment in that year, giving a comprehensive understanding of the shark's investment distribution and impact. 


In [ ]:
#delimiter //
#create procedure getseason(in season int, in sname varchar(255))
#begin
#	case
#			when sname="Namita" then
#            set @tot=(select sum(`Namita_Investment_Amount_in_lakhs`) from sharktank where Season_Number=season);
#            select Industry, sum(`Namita_Investment_Amount_in_lakhs`) as total,(sum(`Namita_Investment_Amount_in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Namita_Investment_Amount_in_lakhs`>0
#            group by Industry;
#            when sname="Vineeta" then
#            set @tot=(select sum(`Vineeta_Investment_Amount_in_lakhs`) from sharktank where Season_Number=season);
#           select Industry, sum(`Vineeta_Investment_Amount_in_lakhs`) as total, (sum(`Vineeta_Investment_Amount_in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Vineeta_Investment_Amount_in_lakhs`>0
#            group by Industry;
#            when sname="Anupam" then
#           set @tot=(select sum(`Anupam_Investment_Amount_in_lakhs`) from sharktank where Season_Number=season);
#            select Industry, sum(`Anupam_Investment_Amount_in_lakhs`) as total, (sum(`Anupam_Investment_Amount_in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Anupam_Investment_Amount_in_lakhs`>0
#            group by Industry;
#            when sname="Aman" then
#            set @tot=(select sum(`Aman_Investment_Amount_in_lakhs`) from sharktank where Season_Number=season);
#            select Industry, sum(`Aman_Investment_Amount_in_lakhs`) as total, (sum(`Aman_Investment_Amount_in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Aman_Investment_Amount_in_lakhs`>0
#            group by Industry;
#            when sname="Peyush" then
#            set @tot=(select sum(`Peyush_Investment_Amount__in_lakhs`) from sharktank where Season_Number=season);
#            select Industry, sum(`Peyush_Investment_Amount__in_lakhs`) as total, (sum(`Peyush_Investment_Amount__in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Peyush_Investment_Amount__in_lakhs`>0
#            group by Industry;
#            when sname="Amit" then
#            set @tot=(select sum(`Amit_Investment_Amount_in_lakhs`) from sharktank where Season_Number=season);
#            select Industry, sum(`Amit_Investment_Amount_in_lakhs`) as total, (sum(`Amit_Investment_Amount_in_lakhs`)/@tot)*100 as perc from sharktank where Season_Number=season and `Amit_Investment_Amount_in_lakhs`>0
#            group by Industry;
#            when sname="Ashneer" then
#            set @tot=(select sum(`Ashneer_Investment_Amount`) from sharktank where Season_Number=season);
#            select Industry, sum(`Ashneer_Investment_Amount`) as total, (sum(`Ashneer_Investment_Amount`)/@tot)*100 as perc from sharktank where Season_Number=season and `Ashneer_Investment_Amount`>0
#            group by Industry;
#            else 
#            select "This is incorrect input";
#    end case ;
#end // 

In [12]:
pd.read_sql_query('call getseason(2, "Anupam")',engine)

,Industry,total,perc
0,Food,240.00,26.234382
1,Electronics,50.00,5.465496
2,Beauty/Fashion,260.00,28.420581
3,Liquor/Beverages,12.50,1.366374
4,Manufacturing,56.50,6.176011
5,Services,45.00,4.918947
6,Vehicles/Electrical Vehicles,33.33,3.643300
7,Technology/Software,70.00,7.651695
8,Furnishing/Household,50.00,5.465496
9,Medical/Health,97.50,10.657718
